In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem
from copy import deepcopy
from asymopt.geom import build_metal_sub_lig_comp_mol
from asymopt.desc import maxminscale,process_desc
from asymopt.ml import model_delta_pred_virt
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
com_df = pd.read_csv('../data/processed/com_data.csv')
lit_df = pd.read_csv('../data/processed/lit_data_with_cplex_smi.csv')
group2smi_map = np.load('../ligand_pool/vir_lig_smi_map.npy',allow_pickle=True).item()
smi2group_map = {v:k for k,v in group2smi_map.items()}
vir_lig_group_map = np.load('../ligand_pool/vir_lig_smi_map.npy',allow_pickle=True).item()
vir_lig_comp_map = np.load('../ligand_pool/vir_lig_comp_map.npy',allow_pickle=True).item()
vir_lig2comp_map = {smi1:smi2 for smi1,smi2 in vir_lig_comp_map.values()}
vir_lig_sas_inf = np.load('../ligand_pool/smi_sascore_map.npy',allow_pickle=True).item()
vir_lig_smi_lst = list(vir_lig_group_map.values())
sel_vir_lig_smi_lst = [smi for smi,score in vir_lig_sas_inf.items() if score < 4]
sel_vir_com_smi_lst = [vir_lig2comp_map[smi] for smi in sel_vir_lig_smi_lst]


com_rct_1_smi,com_rct_2_smi,com_metal = com_df["rct_1_smi"].to_list()[0], com_df["rct_2_smi"].to_list()[0], com_df["metal"].to_list()[0]
com_lig_smi_lst = com_df["lig_smi"].to_list()
com_complex_lst = [build_metal_sub_lig_comp_mol(com_rct_1_smi,com_rct_2_smi,com_metal,lig_smi) for lig_smi in tqdm(com_lig_smi_lst)]
com_complex_smi_lst = [Chem.MolToSmiles(mol) for mol in com_complex_lst]
com_fp = np.load('../descriptor/com_complex_fp_lst.npy',allow_pickle=True).item()
com_rdkit = np.load('../descriptor/com_complex_rdkit_lst.npy',allow_pickle=True).item()

com_fp_arr = np.array([com_fp[smi] for smi in com_complex_smi_lst])
com_rdkit_arr = np.array([com_rdkit[smi] for smi in com_complex_smi_lst])
com_ddG = com_df["ddG"].to_numpy()

lit_ddG = lit_df["ddG"].to_numpy()
lit_fp_arr = np.load("../descriptor/lit_complex_fp_lst.npy")
lit_rdkit_arr = np.load("../descriptor/lit_complex_rdkit_lst.npy")

vir_fp = np.load("../descriptor/vir_complex_fp_lst.npy",allow_pickle=True).item()
vir_rdkit = np.load("../descriptor/vir_complex_rdkit_lst.npy",allow_pickle=True).item()
sel_vir_fp_arr = np.array([vir_fp[smi] for smi in sel_vir_com_smi_lst])
sel_vir_rdkit_arr = np.array([vir_rdkit[smi] for smi in sel_vir_com_smi_lst])
sel_vir_sas_score = np.array([vir_lig_sas_inf[smi] for smi in sel_vir_lig_smi_lst])
sel_vir_sas_score_std = (4-sel_vir_sas_score)/(4-3.5)
all_fp = maxminscale(process_desc(np.concatenate((com_fp_arr,lit_fp_arr,sel_vir_fp_arr),axis=0)))
all_rdkit = maxminscale(process_desc(np.concatenate((com_rdkit_arr,lit_rdkit_arr,sel_vir_rdkit_arr),axis=0)))

com_fp_arr_scaled = all_fp[:len(com_fp_arr)]
com_rdkit_arr_scaled = all_rdkit[:len(com_rdkit_arr)]
lit_fp_arr_scaled = all_fp[len(com_fp_arr):len(com_fp_arr)+len(lit_fp_arr)]
lit_rdkit_arr_scaled = all_rdkit[len(com_rdkit_arr):len(com_rdkit_arr)+len(lit_rdkit_arr)]
sel_vir_fp_arr_scaled = all_fp[len(com_fp_arr)+len(lit_fp_arr):]
sel_vir_rdkit_arr_scaled = all_rdkit[len(com_rdkit_arr)+len(lit_rdkit_arr):]

com_simi_desc_map = {"MF":deepcopy(com_fp_arr_scaled),"RDKit":deepcopy(com_rdkit_arr_scaled)}  
lit_simi_desc_map = {"MF":deepcopy(lit_fp_arr_scaled),"RDKit":deepcopy(lit_rdkit_arr_scaled)}
sel_vir_simi_desc_map = {"MF":deepcopy(sel_vir_fp_arr_scaled),"RDKit":deepcopy(sel_vir_rdkit_arr_scaled)}

com_desc_map = {"MF": com_fp_arr_scaled, "RDKit": com_rdkit_arr_scaled}
lit_desc_map = {"MF": lit_fp_arr_scaled, "RDKit": lit_rdkit_arr_scaled}
sel_vir_desc_map = {"MF": sel_vir_fp_arr_scaled, "RDKit": sel_vir_rdkit_arr_scaled}

model = DecisionTreeRegressor(random_state=0)
lib_p = model_delta_pred_virt(model,
                            base_x=lit_desc_map["MF"],base_y=lit_ddG,
                            delta_x=com_desc_map["MF"],delta_y=com_ddG,
                            tgt_x=sel_vir_desc_map["MF"],
                            dist_type="Euclidean",topk=0.4,
                            tgt_simi_desc=sel_vir_simi_desc_map["RDKit"],
                            base_simi_desc=lit_simi_desc_map["RDKit"])
lib_p_std = maxminscale(lib_p)
complex_score = 0.7 * lib_p_std + 0.3 * sel_vir_sas_score_std
lig_score_map = sorted([[lig,score] for lig,score in zip(sel_vir_lig_smi_lst,complex_score)],key=lambda x:x[1],reverse=True)


100%|██████████| 21/21 [00:00<00:00, 332.54it/s]


[INFO] There are 1221 time(s) different delta prediciton


In [3]:
lig_score_map[:10]

[['CC(C)(C)[S@@](=O)N(Cc1ccccc1)[C@H](CP(c1ccccc1)c1ccccc1)c1ccccc1',
  0.984507933500655],
 ['Cc1ccc([C@@H](CP(c2ccccc2)c2ccccc2)N(Cc2ccccc2)[S@](=O)C(C)(C)C)cc1',
  0.9586246560918823],
 ['Cc1ccc(CN([C@H](CP(c2ccccc2)c2ccccc2)c2ccccc2)[S@](=O)C(C)(C)C)cc1',
  0.9164596608641469],
 ['Cc1ccc(CN([C@H](CP(c2ccc(C)cc2)c2ccc(C)cc2)c2ccccc2)[S@](=O)C(C)(C)C)cc1',
  0.8065228834915739],
 ['COc1ccc(CN([C@H](CP(c2ccccc2)c2ccccc2)c2ccsc2)[S@](=O)C(C)(C)C)cc1',
  0.8022901911105984],
 ['COc1ccc(CN([C@H](CP(c2ccc(C)cc2)c2ccc(C)cc2)c2ccco2)[S@](=O)C(C)(C)C)cc1',
  0.7910572825896576],
 ['CC(C)(C)[S@@](=O)N(Cc1ccccc1)[C@@H](Cc1ccccc1)CP(Cc1ccccc1)Cc1ccccc1',
  0.763714776653071],
 ['Cc1ccc([C@@H](CP(c2ccc(C)cc2)c2ccc(C)cc2)N(Cc2ccccc2)[S@](=O)C(C)(C)C)cc1',
  0.7358077912272822],
 ['Cc1ccc(CN([C@H](CP(Cc2ccccc2)Cc2ccccc2)c2ccc(C)cc2)[S@](=O)C(C)(C)C)cc1',
  0.7263277494696063],
 ['Cc1ccc(CN([C@H](CP(c2ccccc2)c2ccccc2)c2cc(C)cc(C)c2)[S@](=O)C(C)(C)C)cc1',
  0.7231290607716094]]